In [20]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

import re
import numpy as np
import json

In [38]:
groups_file = open('../data/groups', '+r')
artists = groups_file.readlines()

artists = list(map(str.strip, artists))
artists

['Kaleo', 'Måneskin', 'Coldplay']

In [22]:
def get_all_website_links(url):
    """
    Returns all URLs that is found on `url` in which it belongs to the same website
    """
    # all URLs of `url`
    urls = []
    # domain name of the URL without the protocol
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # href empty tag
            continue
                # join the URL if it's relative (not absolute link)
        if href.find('lyric/')>0:
            href = urljoin(url, href)
            parsed_href = urlparse(href)
            # remove URL GET parameters, URL fragments, etc.
            href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
            urls.append(href)
        
    return urls

In [23]:
def get_lyric(url) -> list:

    # domain name of the URL without the protocol
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")

    # gets a tag with a song lyrics

    tag = soup.find("pre", id="lyric-body-text")
    
    # Search for a name of the song with a pattern
    try:
        name_song_expression = '\-\s(.*)\sLyrics\s\|'
        name_song = re.findall(name_song_expression, soup.get_text())[0]
    except:
        print("Song name for song is not available")
        return 0
    try: 
        all_text = tag.get_text()
    except:
        print("Lyric for song is not available")
        all_text = "Lyric for song is not available"
    return {name_song:all_text}

In [24]:
# ARCHIVE VERSION

def get_lyric_archive(url) -> dict:
   
    # domain name of the URL without the protocol
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    # gets a tag with a song lyrics

    comments = soup.find('div', class_='col-xs-12')

    # Search for a name of the song with a pattern
    try: 
        name_song_expression = '\"(.*)\" lyrics'
        name_song = re.findall(name_song_expression, soup.get_text())[0]
    except:
        print("Song_name for song is not available")
        return 0
    try: 
        #for item in comments:
        all_text = comments.get_text()
        
        sp='KALEO\slyrics\"(.*)Submit\stext'
        print(sp)
        song_text = re.search(r'KALEO\sLyrics(.*)Submit', all_text, re.DOTALL)
        song_text.group(0)
    #find string find_all_next
    except:
        print("Lyric for song is not available")
        song_text = "Lyric for song is not available"
    return [name_song,song_text.group(0)]

In [25]:
def grab_artist(art):
    '''
    ArithmeticError
    '''
    header={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:108.0) Gecko/20100101 Firefox/108.0'}
    url='https://www.lyrics.com/artist/' + art

    response=requests.get(url, headers=header)
    lyric_url = get_all_website_links(url)
    # Delete after a check
    print(len(lyric_url))
    print(lyric_url)

    #print links and grab them into a dictionary
    songs=dict()
    for item in lyric_url:
        print(item)
        result = get_lyric(item)
        if result:
            songs.update(result)

    #just check
    for item in songs:
        print(item)
    
    json_name = art + '.json'
    with open(json_name, 'w') as fp:
        json.dump(songs, fp)
        fp.close()

    return json_name + 'is successfully uploaded'



In [26]:
for artist in artists:
    print(grab_artist(artist))



19
['https://www.lyrics.com/lyric/36786545/Kaleo/I+Want+More', 'https://www.lyrics.com/lyric/36110819/Kaleo/Way+Down+We+Go', 'https://www.lyrics.com/lyric/36786544/Kaleo/Break+My+Baby', 'https://www.lyrics.com/lyric/36963289/Kaleo/Alter+Ego', 'https://www.lyrics.com/lyric/37383272/Kaleo/All+The+Pretty+Girls', 'https://www.lyrics.com/lyric/37041613/Kaleo/Backbone', 'https://www.lyrics.com/lyric/38252185/Kaleo/Hey+Gringo', 'https://www.lyrics.com/lyric/38252181/Kaleo/I+Walk+on+Water', 'https://www.lyrics.com/lyric/38252184/Kaleo/My+Fair+Lady', 'https://www.lyrics.com/lyric/38252187/Kaleo/Free+the+Slave', 'https://www.lyrics.com/lyric/38252190/Kaleo/Brother+Run+Fast', 'https://www.lyrics.com/lyric/38252186/Kaleo/Skinny', 'https://www.lyrics.com/lyric/38252182/Kaleo/Backbone', 'https://www.lyrics.com/lyric/38252183/Kaleo/I+Want+More', 'https://www.lyrics.com/lyric/38252189/Kaleo/Break+My+Baby', 'https://www.lyrics.com/lyric/38252180/Kaleo/Into+My+Mother%27s+Arms', 'https://www.lyrics.com/s